<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/team4_movie_analysis/movie_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [52]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, to_date, year

spark = SparkSession.builder.appName("Movies-Activity").getOrCreate()

df_movies = spark.read \
            .option("inferSchema", "true") \
            .json("./movies.json")

df_movies.show(n=7)



+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|      The Land Girls|        NULL|  146083|         146083|


In [23]:
# Get Profit margin
df_movies = df_movies.withColumn('Profit_Margin', (df_movies.Worldwide_Gross - df_movies.Production_Budget) / df_movies.Production_Budget)

df_movies.show(n=7)


+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|      Profit_Margin|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|

In [24]:
# Movie is hit or not
df_movies = df_movies.withColumn('Box_Office_Hit',  when(df_movies.Worldwide_Gross > df_movies.Production_Budget * 2, 'Hit').otherwise('Flop'))

df_movies.show(n=7)


+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+--------------+
|       Creative_Type|         Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|      Profit_Margin|Box_Office_Hit|
+--------------------+-----------------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+-------------------+--------------+
|                NULL|             NULL|   Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|               

In [46]:
df_movies = df_movies.withColumn('Release_Year', year(to_date(df_movies.Release_Date, 'd-MMM-yy')) - 100)

df_movies.show(n=10)


+--------------------+-----------------+-------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+------------+
|       Creative_Type|         Director|  Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Release_Year|
+--------------------+-----------------+-------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+------------+
|                NULL|             NULL|     Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|                  NULL|            NULL|               NULL|      The Lan

In [48]:
df_movies = df_movies.withColumn('IMDB_Rating_Category', when(df_movies.IMDB_Rating >= 7, 'High').when(df_movies.IMDB_Rating >= 5, 'Medium').otherwise('Low'))

df_movies.show(n=10)


+--------------------+-----------------+-------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+------------+--------------------+
|       Creative_Type|         Director|  Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|             Source|               Title|US_DVD_Sales|US_Gross|Worldwide_Gross|Release_Year|IMDB_Rating_Category|
+--------------------+-----------------+-------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------+------------+--------+---------------+------------+--------------------+
|                NULL|             NULL|     Gramercy|        6.1|      1071|          R|       NULL|          8000000|   12-Jun-98|          

In [53]:
df_movies_new = df_movies.groupBy(df_movies.Distributor).avg('IMDB_Rating')

df_movies_new.show(n=10)

+--------------------+------------------+
|         Distributor|  avg(IMDB_Rating)|
+--------------------+------------------+
|Oscilloscope Pict...|               6.2|
|               Savoy| 6.533333333333334|
|             Embassy|              NULL|
|         Fader Films|               6.5|
|       October Films|               6.4|
|              Strand| 6.322222222222223|
|             Trimark| 5.085714285714286|
|              Matson|               7.3|
|      Cinema Service|               7.1|
| Weinstein/Dimension|5.7250000000000005|
+--------------------+------------------+
only showing top 10 rows



In [54]:
spark.stop()